## Information Extraction Using Spacy

### Importing Libraries

In [1]:
import glob
import pandas as pd
import re
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt

In [2]:
import os
import json

### Reading all Files using glob

In [3]:
list_of_files = glob.glob("Data/*.txt")

### Creating grammers for rule based Information Extrcation using SPACY

In [196]:
g1 = [{'POS' : {'IN':['NOUN','PROPN']}},
          {'POS':'PUNCT', 'OP':'?'},
          {'POS': {'IN':['VERB', 'NOUN']}},
          {'POS' : 'VERB'},
          {'POS':'ADJ'},
          {'POS':'ADP'},
          {'POS' : 'ADJ' , 'OP': '?'},
          {'POS' : 'ADP' , 'OP':'?'},
          {'POS':'NUM'},
          {'POS':'NOUN' , 'OP' : '?'},
          {'LOWER':'(', 'OP' : '?'},
          {'POS':{'IN':['NUM', 'PUNCT']}},
        {'LOWER': ')' , 'OP':'?'}]
          
g2 = [{'POS':'ADJ'},{'POS':'NOUN'},{'POS':'ADJ'},{'POS':'ADJ'},{'POS':'NOUN'},{'POS':"PUNCT",'OP':'?'},{'POS':'NUM'},{'POS':'PUNCT','OP':'?'}]

g3 = [{'POS' : 'ADJ'},
     {'POS':'ADJ'},
     {'POS':'NOUN'},
     {'POS':'PUNCT'},
     {'POS':'NOUN'},
     {'POS' : 'PUNCT'},
     {'LOWER':'(','OP':'?'},
     {'POS':{'IN':['NUM', 'PUNCT']}},
     {'LOWER':')','OP':'?'}]

g4 = [{'POS': {"IN":['NOUN', 'PROPN']}},
     {'POS':'CCONJ'},
     {'POS':'NOUN'},
     {'POS' : 'PUNCT' , 'OP': '?'},
     {'POS':{'IN':['NUM', 'PUNCT']}},
     {'POS' : 'PUNCT' , 'OP': '?'}]


g13 = [{'POS':'NOUN'},
      {'POS':'ADP'},
      {'POS':'NOUN'},
      {'POS': 'CCONJ'},
      {'DEP':{'IN':['prep', 'conj']}},
      {'POS':'NOUN'},
    {'POS' : 'PUNCT' , 'OP': '?'},
      {'POS':{'IN':['NUM', 'PUNCT']}},
      {'POS' : 'PUNCT' , 'OP': '?'}]

g10 = [{'POS' : {'IN':['ADJ', 'VERB','PROPN']}},
      {'POS':'NOUN'},
      {'ORTH':{'IN': ['capital', 'Capital']}}]

g12 = [{'POS': 'NUM'},
    {'POS':{'IN':['ADJ','PROPN','NOUN']}},
      {'POS':{'IN': ['PROPN', 'NOUN']}},
       {'POS':'ADP','OP':'?'},
       {'POS':'NOUN','OP':'?'},
      {'POS':{'IN':['NUM','PROPN', 'ADV']}},
      {'POS':'ADP','OP':'?'},
      {'POS':'NOUN','OP':'?'},
      {'POS':'DET'},
      {'POS':{'IN':['NUM', 'PUNCT']}}]


g11 = [{'POS':{'IN':['PROPN', 'ADJ']}},
      {'POS':  {'IN':['PROPN', 'NOUN']}},
      {'POS':'NOUN'},
       {'LOWER': '(', 'OP': '?'},
      {'POS':{'IN':['NUM', 'PUNCT']}},
      {'LOWER': '(','OP': '?'}]

g0 = [{'ORTH':{'IN':['Current','Fixed','Net']}},
   {'ORTH': {'IN':['Assets','assets','asset','Asset']}},
   {'POS': {'IN':['NUM','SPACE']}, 'OP': '?'}]

g5 = [{'POS' : 'VERB'},
{'POS' : 'PART'},
{'POS' : 'NOUN'}, 
{'POS' : 'NOUN'},
{'POS' : 'ADV'},
{'POS' : 'VERB'},
{'POS' :  {'IN' : ['NUM' ,'PUNCT']}}]


g6 = [{'POS': {'IN':['NOUN', 'PROPN']}},
{'POS' : 'CCONJ'},
{'POS' : {'IN':['VERB','ADJ']}},
{'POS' :  'NOUN'},
{'LOWER': '(', 'OP':'?'},
{'POS': {'IN' : ['NUM','PUNCT']}},
{'LOWER': ')', 'OP': '?'}]

 
g7 = [{'POS':{'IN':['NOUN','PROPN']}},
{'LOWER' :'for'},
{'POS': 'NOUN'},
{'LOWER': '(', 'OP': '?'},
{'POS': {'IN':['NUM','SPACE', 'PUNCT']}},
{'LOWER': ')', 'OP': '?'}]


g14 = [{'ORTH': 'Net'},
      {'ORTH': {'IN': ['Current', 'current']}, 'OP':'?'},
      {'ORTH': {'IN':['Assets', 'assets','asset','Asset','liabilities' ]}},
        {'LOWER': '(', 'OP':'?'},
      {'POS': 'NUM'},
        {'LOWER': ")", 'OP':'?'}]

g15 = [{'POS':'ADJ'},
      {'POS': 'NOUN'},
      {'LOWER': 'of'},
      {'POS': 'NOUN'},
      {'POS': 'NUM'}]
g16 = [{'ORTH': {"IN":['Reserves','reserves'] }},
      {'POS': 'NUM','OP': '?'}]

grammer = [g0,g1,g2,g3,g4,g5,g6,g7,g10,g11,g12,g13,g14,g15,g16]

### Creating  Functions for Usability

In [190]:
# find the region from where need to extract
def annotating_text(text):
    line_no = []
    for i, line in enumerate(text.splitlines()): 
        word_list = line.split()
        if 'STATEMENTS' in word_list or 'For' in word_list[:4] or 'company' in word_list[:10]:
            line_no.append(i)
    return line_no
# checking for financila year 2019
def check_for_financial_year(text):
    flag = False
    for i , line in enumerate(text.splitlines()):
        word_list = line.split()
        if '2019' in word_list and i < 3:
            flag = True
    return flag
# extracting fields using rule based information extraction
def rule_based_matching(text , grammer):
    extracted_list = []
    for i , line in enumerate(text.splitlines()):
        word_list = line.split()
        if i < min(line_no):
            item = ' '.join(word_list)
            doc = nlp(item)
            matcher = Matcher(nlp.vocab)
            for i in grammer:
                 #print(i)
                matcher.add("matching_1", None, i) 
                 #print()
            matches = matcher(doc) 
            if matches:
                for i in matches:
                    span = doc[i[1]:i[2]] 
                    extracted_list.append(span.text)
    return extracted_list 
# substituting currency symbols with HTML codes
def substitute_currency(key):
    #print("before" , key)
    key = re.sub("Â$","&#36", key)
    key = re.sub("Â€", "&#8364", key)
    key = re.sub("Â£", "&#163", key) 
    #print("after" , key)
    return key

# creating dictionary of fields
def get_dictionary_of_fields(extracted_list , flag):
    dict_ = {}
    if flag == True:
        for i in extracted_list:
                f = i.split()
                if re.findall('\d+|-',f[-1]):
                    k = " ".join(f[:-1])
                    key = substitute_currency(k)
                    if "(" in f[-1]:
                        v1 =  re.sub("\(" , '-' , f[-1])
                        v = re.sub("\)" , '' , v1)
                        dict_.update({key:v})
                    else:
                        dict_.update({key:f[-1]})
                else:
                    k = " ".join(f)
                    key = substitute_currency(k)
                    dict_.update({key:"nan"})
        return (json.dumps(dict_))
    else:
        for i in extracted_list:
            f = i.split()
            if re.findall('\d+|-',f[-1]):
                k = " ".join(f[:-1])
                key = substitute_currency(k)
                dict_.update({key:"nan"})
            else:
                k = " ".join(f)
                key = substitute_currency(k)
                dict_.update({key:"nan"})
        return (json.dumps(dict_))

In [197]:
all_data = []

### Running for all 500 files

In [198]:
for file in list_of_files:
    final_dict = {}
    flag = False
    with open(file)as f:
        text = f.read() 
        filename = os.path.basename(file)
        print(filename)
        line_no = annotating_text(text)
        flag = check_for_financial_year(text)
        extracted_fields = rule_based_matching(text , grammer)
        dict_ = get_dictionary_of_fields(extracted_fields , flag)
        final_dict.update({"Extracted Values":(dict_)})
        final_dict.update({"Filename":os.path.splitext(filename)[0]})
        all_data.append(final_dict)
    #print(final_dict)

X8XUG1JD.txt
X8XX000W.txt
X8XX00J6.txt
X8XX00LL.txt
X8XX00MX.txt
X8XX00YW.txt
X8XX0137.txt
X8XX0196.txt
X8XX01EH.txt
X8XX01EP.txt
X8XX01IX.txt
X8XX01SZ.txt
X8XX01TN.txt
X8XX01TV.txt
X8XX02AX.txt
X8XX02N7.txt
X8XX02SY.txt
X8XX036G.txt
X8XX03IJ.txt
X8XX03S9.txt
X8XX07H7.txt
X8XX0ANN.txt
X8XX0BU1.txt
X8XX0C57.txt
X8XX0CEA.txt
X8XZI954.txt
X8XZI9MH.txt
X8XZI9Y0.txt
X8XZIB3U.txt
X8XZIBDK.txt
X8XZIBOZ.txt
X8XZIBXE.txt
X8XZIBZD.txt
X8XZICIB.txt
X8XZICNE.txt
X8XZICQX.txt
X8XZIDG2.txt
X8XZIDMH.txt
X8XZIE17.txt
X8XZIEI1.txt
X8XZIF16.txt
X8XZIFE8.txt
X8XZIFNV.txt
X8XZIFZL.txt
X8XZIG2X.txt
X8XZIGW8.txt
X8XZIIRN.txt
X8XZIIW2.txt
X8XZIIYQ.txt
X8XZIJC8.txt
X8XZIKDF.txt
X8XZIKJ6.txt
X8XZIKOX.txt
X8XZIMDT.txt
X8XZIMKG.txt
X8XZIML4.txt
X8XZIMQ3.txt
X8XZIN35.txt
X8XZINN6.txt
X8XZINW9.txt
X8XZIO5G.txt
X8XZIOFU.txt
X8XZIOKY.txt
X8XZIOV4.txt
X8XZIOXK.txt
X8XZIP1V.txt
X8XZIPBM.txt
X8XZIPGP.txt
X8XZIPWZ.txt
X8XZIQ16.txt
X8XZIQEG.txt
X8XZIR1E.txt
X8XZIR3U.txt
X8XZIR7V.txt
X8XZIRI8.txt
X8XZIS17.txt
X8XZIS4B.txt

### storing all data

In [199]:
df = pd.DataFrame(all_data)

In [200]:
df = df[['Filename' , 'Extracted Values']]

In [201]:
df.to_csv("Results.csv")